In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import json
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
with open('/kaggle/input/nlp-ass4-dataset/val_file.json', 'r') as val_file:
    val_data = json.load(val_file)
test_texts = [item["utterances"] for item in val_data]
test_labels = [item["emotions"] for item in val_data]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model = bert_model.to(device)

In [ ]:
labels_dict = {'anger'  :1,
               'joy'    :2,
               'fear'   :3,
               'disgust':4,
               'neutral':5,
               'surprise':6,
               'sadness':7
              }

inverse_dict = {}
for i in labels_dict:
    inverse_dict[labels_dict[i]] = i
inverse_dict[0] = 'neutral'

In [ ]:
converted_test_inputs = []
converted_test_labels = []

for i in test_labels:
    temp = []
    for j in i:
        temp.append(labels_dict[j])
    converted_test_labels.append(temp)

for i, j in enumerate(test_texts):
    temp_len = len(test_texts[i])
    if(temp_len<25):
        for lol in range(25-temp_len):
            test_texts[i].append("")
            converted_test_labels[i].append(0)
            
    inputs = tokenizer(test_texts[i], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    converted_test_inputs.append(outputs[1])

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
        self.fc3 = nn.Linear(hidden_size//4, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_size = 768
hidden_size = 256
output_size = 8

model = LSTMModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('/kaggle/input/gru/pytorch/lstm_nlp4/1/lstm.pth'))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.eval()
text = ["You-you\u0085you had sex with Ursula?!","Uh, a little bit. She-she-she walked in and I thought she was you and I kissed her and","You didn't notice she was wearing different clothes?!","Well I was just so excited to see you.","Oh. Ew! Ew! Ew! Ugh! Y'know what? This is too weird."]
temp_len = len(text)
if(temp_len<25):
    for lol in range(25-temp_len):
        text.append("")

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
inputs = {key: val.to(device) for key, val in inputs.items()}
with torch.no_grad():
    outputs = bert_model(**inputs)

output = model(outputs[1])
final_result = []
for i in range(temp_len):
    x = output[i]
    probabilities = torch.nn.functional.softmax(x, dim=0)
    predicted_class = torch.argmax(probabilities).item()
    final_result.append(inverse_dict[predicted_class])
print(final_result)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, hidden_size//4)
        self.fc3 = nn.Linear(hidden_size//4, output_size)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc1(out)
        out = torch.relu(out)
        out = self.fc2(out)
        out = torch.relu(out)
        out = self.fc3(out)
        return out

input_size = 768
hidden_size = 256
output_size = 8

model = GRUModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('/kaggle/input/gru/pytorch/gru_nlp4/1/gru.pth'))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
label_list = []
final_list = []
for i in range(len(test_texts)):
    inputs = converted_test_inputs[i]
    labels = converted_test_labels[i]
    label_list.append(labels)
    zero_c = labels.count(0)
    output = model(inputs.to(device))
    temp_inputs = [x for x in test_texts[i] if x!=""]
    for j in range(len(inputs)+1):
        x = output[j]
        probabilities = torch.nn.functional.softmax(x, dim=0)
        predicted_class = torch.argmax(probabilities).item()
        final_list.append(predicted_class)
#         print(predicted_class)
print(len(label_list))
print(len(final_list))

        

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(label_list, final_list)
print("Accuracy:", accuracy)
